# KG 성능 비교 실험 (ROUGE 기반 평가)


In [1]:
import pandas as pd
from rouge_score import rouge_scorer

# 데이터 불러오기
kg = pd.read_csv("kg_triples_test.csv")
test_df = pd.read_csv("test.csv")

# 트리플 결합 텍스트 만들기
kg["triple_text"] = kg["subject"] + " " + kg["predicate"] + " " + kg["object"]


In [2]:
# 간단한 유사도 기반 KG 검색 함수
def find_relevant_kg(text, kg_texts, top_k=3):
    matches = []
    for t in kg_texts:
        score = sum([1 for w in t.split() if w in text])
        matches.append((t, score))
    matches.sort(key=lambda x: x[1], reverse=True)
    return [x[0] for x in matches[:top_k]]


In [3]:
# 프롬프트 구성 함수
def build_prompt(text, kg_hits):
    context = "\n".join(kg_hits)
    return f"다음은 참고 지식입니다:\n{context}\n\n사용자 입력: {text}\n답변:"


In [4]:
# ROUGE 평가 함수
def evaluate_with_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = [scorer.score(ref, pred)["rougeL"].fmeasure for pred, ref in zip(predictions, references)]
    return sum(scores) / len(scores)


In [5]:
from random import randint

# 샘플 10개 추출
sample = test_df.sample(10, random_state=42)

# 1. KG 없이 (baseline)
preds_no_kg = ["답변: " + text[:20] for text in sample["input"]]

# 2. KG 사용
preds_with_kg = []
for text in sample["input"]:
    hits = find_relevant_kg(text, kg["triple_text"])
    prompt = build_prompt(text, hits)
    preds_with_kg.append("답변: " + text[:20] + " (지식반영)")

# 평가
rouge_no_kg = evaluate_with_rouge(preds_no_kg, sample["output"])
rouge_with_kg = evaluate_with_rouge(preds_with_kg, sample["output"])

print(f"KG 미사용 시 ROUGE-L: {rouge_no_kg:.4f}")
print(f"KG 사용 시 ROUGE-L: {rouge_with_kg:.4f}")


KeyError: 'input'

In [18]:
# ✅ 1. 필요한 라이브러리 로딩
import pandas as pd
import subprocess

# ✅ 2. Knowledge Graph 로딩 및 텍스트 구성
kg = pd.read_csv("kg_triples_test.csv")
kg["triple_text"] = kg["subject"] + " " + kg["predicate"] + " " + kg["object"]

# ✅ 3. 테스트 질문 로딩
test_df = pd.read_csv("test.csv")

# ✅ 4. KG 검색 함수
def find_relevant_kg(user_input, kg_texts, topk=3):
    return [t for t in kg_texts if any(word in user_input for word in t.split())][:topk]

# ✅ 5. 프롬프트 생성 함수
def build_prompt(user_input, kg_hits=None):
    prompt = f"질문: {user_input}\n"
    if kg_hits:
        prompt += "배경지식:\n"
        for hit in kg_hits:
            prompt += f"- {hit}\n"
    prompt += "답변:"
    return prompt

# ✅ 6. 로컬 Ollama 모델 호출 함수 (예: llama3)
def run_ollama(prompt, model="llama3"):
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        timeout=30
    )
    return result.stdout.decode("utf-8").strip()

# ✅ 7. 질문 입력 → KG 사용 여부에 따른 비교
sample = test_df.sample(1, random_state=42)
user_input = sample["sentence1"].values[0]
true_answer = sample["sentence2"].values[0]

kg_hits = find_relevant_kg(user_input, kg["triple_text"])
prompt_kg = build_prompt(user_input, kg_hits)
prompt_no_kg = build_prompt(user_input)

print("📌 질문:", user_input)
print("\n[프롬프트 - KG 미사용]\n", prompt_no_kg)
print("\n[프롬프트 - KG 사용]\n", prompt_kg)

# ✅ 8. Ollama 실행 및 출력 결과
print("\n🤖 [KG 미사용 답변]\n")
print(run_ollama(prompt_no_kg))

print("\n🤖 [KG 사용 답변]\n")
print(run_ollama(prompt_kg))

📌 질문: 라이스 페이퍼의 두께나 구성이 일반적이지 않아 조금 놀랐습니다. 아마, 식사량을 충분히 하는 것이 목표였을건데, 이로 인해 월남쌈의 질감이나 맛을 느끼는 부분에서 조금 힘들었습니다. 개인적으로는 훠궈 육수는 조금 아쉬움이 남았지만, 다른 메뉴들 중에서는 고기가 빼어난 맛을 보였습니다.

[프롬프트 - KG 미사용]
 질문: 라이스 페이퍼의 두께나 구성이 일반적이지 않아 조금 놀랐습니다. 아마, 식사량을 충분히 하는 것이 목표였을건데, 이로 인해 월남쌈의 질감이나 맛을 느끼는 부분에서 조금 힘들었습니다. 개인적으로는 훠궈 육수는 조금 아쉬움이 남았지만, 다른 메뉴들 중에서는 고기가 빼어난 맛을 보였습니다.
답변:

[프롬프트 - KG 사용]
 질문: 라이스 페이퍼의 두께나 구성이 일반적이지 않아 조금 놀랐습니다. 아마, 식사량을 충분히 하는 것이 목표였을건데, 이로 인해 월남쌈의 질감이나 맛을 느끼는 부분에서 조금 힘들었습니다. 개인적으로는 훠궈 육수는 조금 아쉬움이 남았지만, 다른 메뉴들 중에서는 고기가 빼어난 맛을 보였습니다.
배경지식:
- 사용자 질문한다 다른 사람의 의도
답변:

🤖 [KG 미사용 답변]



FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다

In [22]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import pandas as pd

# CPU로 강제 지정
device = torch.device("cpu")

# KoBERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForMaskedLM.from_pretrained('monologg/kobert')
model.to(device)
model.eval()

# 사용자 질문 + KG 포함된 프롬프트 만들기
def build_prompt(user_input, kg_hits=None):
    prompt = ""
    if kg_hits:
        prompt += "배경지식:\n"
        for hit in kg_hits:
            prompt += f"- {hit}\n"
    prompt += f"질문: {user_input}"
    return prompt

# KG 불러오기
kg_df = pd.read_csv('kg_triples_test.csv')

# KG에서 관련 배경지식 추출 (간단한 키워드 검색)
def find_relevant_kg(question, kg_df):
    hits = []
    for triple in kg_df['triple_text']:
        if any(word in triple for word in question.split()):
            hits.append(triple)
    return hits[:3]

# KoBERT를 통한 MASK 예측
def generate_answer(prompt):
    if "[MASK]" not in prompt:
        return "⚠️ 질문에 [MASK]를 포함시켜야 합니다."

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits

    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    mask_token_logits = predictions[0, mask_token_index, :]
    top_token = torch.argmax(mask_token_logits, dim=1)
    predicted_token = tokenizer.decode(top_token)
    return predicted_token


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
